### Capstone Project: Predicting Crime in San Francisco

by Elton Yeo, DSI13

#### Problem Statement and Context

Police departments have limited frontline resources, and need to prioritise areas where their officers patrol. Police patrols are important because they project presence, which can deter criminals and increase the sense of safety for residents. 

This project aims to predict the number of crime incidents that will take place in a particular zip code, given a range of variables such as the day of the week and time of the day. The data will be run through 3 regression models: linear regression, random forest, and XGBoost. 

The models will be trained on 2018 data, and tested on 2019 data, and evaluated by their r-sqaured scores. Success is defined as a r-sqaured score of 0.8 and above, which means that the model explains 80% or more of the variability of the target data that is predictable from the independent variables.

(Question: should i do a train-test split on the entire dataset, or as above?)

#### Risks and Assumptions of Data

The data may have been recorded in a manner that is useful for frontline officers or operators, thus impeding data cleaing and our understanding of the data. 

We assume that the data was recorded/provided accurately by the frontline officers or the citizens who had reported the crimes. 

Data source: https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
reports=pd.read_csv('../data/reports.csv')

In [20]:
#standardising column names, replacing the spaces with underscores
#...and converting all letters to lowercase 
reports.columns = reports.columns.str.replace(' ', '_').str.lower()

In [21]:
reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329469 entries, 0 to 329468
Data columns (total 36 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   incident_datetime                                     329469 non-null  object 
 1   incident_date                                         329469 non-null  object 
 2   incident_time                                         329469 non-null  object 
 3   incident_year                                         329469 non-null  int64  
 4   incident_day_of_week                                  329469 non-null  object 
 5   report_datetime                                       329469 non-null  object 
 6   row_id                                                329469 non-null  int64  
 7   incident_id                                           329469 non-null  int64  
 8   incident_number                             

We will remove the following variables:
- incident_datetime: since it repeats information found in incident_date and incident_time
- report_datetime: since this is not critical to the occurrence of the crime incident, and is only an administrative record of when the report of the incident was filed
- row_id: this is an identifier unique to the dataset; we do not need it since we have the index
- incident_id: this is an adminstrative number to keep track of the incidents, and is not critical to the occurrence of the crime incident
- incident_number: this is also an adminstrative number to keep track of incidents, and is not critical to the occurrence of the crime incident
- cad_number: this is an adminstrative number issued by the Dept of Emergency Management and is tagged to the incident_number; it is not critical to the occurrence of the crime incident
- report_type_code: this is a label for the report_type_description, and is not necessary
- incident_code: this is a label for incident_category, and is not necessary
- incident_subcategory: such level of detail is not necesary for the model
- incident_description: such level of detail is not necessary for the model
- intersection: this is the 2 or more street names that intersect closest to the original incident; it is not necessary since we will be using the longitude and latitude for the precise locations
- cnn: unique identifier of the intersection which can be removed
- point: point geometry used for mapping features in the open data portal platform which can be removed

In [61]:
reports.drop(columns=['incident_datetime', 'report_datetime', 'row_id', 
                      'incident_id', 'incident_number', 'cad_number', 
                      'report_type_code', 'incident_code', 'incident_subcategory', 
                      'incident_description', 'intersection', 'cnn', 'point'], inplace=True)

In [62]:
reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329469 entries, 0 to 329468
Data columns (total 23 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   incident_date                                         329469 non-null  object 
 1   incident_time                                         329469 non-null  object 
 2   incident_year                                         329469 non-null  int64  
 3   incident_day_of_week                                  329469 non-null  object 
 4   report_type_description                               329469 non-null  object 
 5   filed_online                                          71410 non-null   object 
 6   incident_category                                     329438 non-null  object 
 7   resolution                                            329469 non-null  object 
 8   police_district                             

In [64]:
reports.head()

,incident_date,incident_time,incident_year,incident_day_of_week,report_type_description,filed_online,incident_category,resolution,police_district,analysis_neighborhood,...,sf_find_neighborhoods,current_police_districts,current_supervisor_districts,analysis_neighborhoods,hsoc_zones_as_of_2018-06-05,owed_public_spaces,central_market/tenderloin_boundary_polygon_-_updated,parks_alliance_cpsi_(27+tl_sites),esncag_-_boundary_file,"areas_of_vulnerability,_2016"
0,2019/05/01,01:00,2019,Wednesday,Initial,NaN,Offences Against The Family And Children,Open or Active,Taraval,Sunset/Parkside,...,39.0,10.0,7.0,35.0,NaN,NaN,NaN,NaN,NaN,1.0
1,2019/06/22,07:45,2019,Saturday,Initial,NaN,Non-Criminal,Open or Active,Southern,South of Market,...,32.0,1.0,10.0,34.0,1.0,NaN,1.0,NaN,NaN,2.0
2,2019/06/03,16:16,2019,Monday,Initial Supplement,NaN,Missing Person,Open or Active,Bayview,Bayview Hunters Point,...,88.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0
3,2018/11/16,16:34,2018,Friday,Initial Supplement,NaN,Offences Against The Family And Children,Cite or Arrest Adult,Central,Chinatown,...,104.0,6.0,3.0,6.0,NaN,18.0,NaN,NaN,NaN,2.0
4,2019/05/27,02:25,2019,Monday,Initial,NaN,Assault,Open or Active,Northern,Marina,...,15.0,4.0,6.0,13.0,NaN,NaN,NaN,NaN,NaN,1.0


In [65]:
reports.isnull().sum()

incident_date                                                0
incident_time                                                0
incident_year                                                0
incident_day_of_week                                         0
report_type_description                                      0
filed_online                                            258059
incident_category                                           31
resolution                                                   0
police_district                                              0
analysis_neighborhood                                    17733
supervisor_district                                      17668
latitude                                                 17668
longitude                                                17668
sf_find_neighborhoods                                    24336
current_police_districts                                 18142
current_supervisor_districts                           

In [31]:
reports[['incident_code', 'incident_category']][reports.incident_category.isnull()]

,incident_code,incident_category
7938,65021,NaN
8536,65021,NaN
8597,65021,NaN
8689,65021,NaN
8758,65021,NaN
11274,65021,NaN
11291,65021,NaN
11416,65021,NaN
12873,65021,NaN
14377,65021,NaN


In [42]:
reports[reports['incident_code'] == 65021]

,incident_datetime,incident_date,incident_time,incident_year,incident_day_of_week,report_datetime,row_id,incident_id,incident_number,cad_number,...,sf_find_neighborhoods,current_police_districts,current_supervisor_districts,analysis_neighborhoods,hsoc_zones_as_of_2018-06-05,owed_public_spaces,central_market/tenderloin_boundary_polygon_-_updated,parks_alliance_cpsi_(27+tl_sites),esncag_-_boundary_file,"areas_of_vulnerability,_2016"
7938,2020/02/16 01:28:00 AM,2020/02/16,01:28,2020,Sunday,2020/02/16 01:34:00 AM,90321865021,903218,200117003,200470190.0,...,85.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
8536,2020/02/16 01:28:00 AM,2020/02/16,01:28,2020,Sunday,2020/02/16 01:34:00 AM,90352465021,903524,200117003,200470190.0,...,85.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
8597,2020/02/16 10:08:00 PM,2020/02/16,22:08,2020,Sunday,2020/02/16 10:08:00 PM,90352265021,903522,200119190,200473367.0,...,90.0,9.0,1.0,7.0,NaN,NaN,NaN,NaN,NaN,2.0
8689,2020/02/16 09:10:00 PM,2020/02/16,21:10,2020,Sunday,2020/02/16 09:12:00 PM,90354465021,903544,200119128,200473185.0,...,88.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
8758,2020/02/16 10:33:00 PM,2020/02/16,22:33,2020,Sunday,2020/02/16 10:33:00 PM,90352365021,903523,200119190,200473459.0,...,90.0,9.0,1.0,7.0,NaN,NaN,NaN,NaN,NaN,2.0
11274,2020/02/23 01:48:00 AM,2020/02/23,01:48,2020,Sunday,2020/02/23 01:52:00 AM,90580065021,905800,200135776,200540222.0,...,115.0,3.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,1.0
11291,2020/02/22 11:59:00 PM,2020/02/22,23:59,2020,Saturday,2020/02/22 11:59:00 PM,90578465021,905784,200135760,200534140.0,...,85.0,2.0,9.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0
11416,2020/02/23 03:36:00 AM,2020/02/23,03:36,2020,Sunday,2020/02/23 03:36:00 AM,90580865021,905808,200135958,200540462.0,...,8.0,8.0,4.0,29.0,NaN,NaN,NaN,NaN,NaN,1.0
12873,2020/02/23 03:36:00 AM,2020/02/23,03:36,2020,Sunday,2020/02/26 11:01:00 AM,90693265021,906932,200135958,200540462.0,...,8.0,8.0,4.0,29.0,NaN,NaN,NaN,NaN,NaN,1.0
14377,2020/03/01 02:51:00 PM,2020/03/01,14:51,2020,Sunday,2020/03/01 02:51:00 PM,90845565021,908455,200155495,200612139.0,...,NaN,8.0,4.0,29.0,NaN,NaN,NaN,NaN,NaN,1.0


In [58]:
reports.incident_category.value_counts()

Larceny Theft                                   106253
Other Miscellaneous                              24506
Non-Criminal                                     20373
Malicious Mischief                               19600
Assault                                          19500
Burglary                                         14315
Warrant                                          12063
Lost Property                                    12042
Motor Vehicle Theft                              11768
Fraud                                             9784
Recovered Vehicle                                 9526
Drug Offense                                      8473
Robbery                                           7825
Missing Person                                    7521
Offences Against The Family And Children          6339
Suspicious Occ                                    6125
Disorderly Conduct                                5816
Traffic Violation Arrest                          4269
Miscellane

In [67]:
reports.analysis_neighborhoods.value_counts()

20.0    36113
36.0    32454
8.0     29377
34.0    27209
1.0     18057
39.0    10317
23.0    10018
5.0      9439
35.0     9071
21.0     8951
9.0      8506
13.0     7490
32.0     7165
29.0     6859
6.0      6358
41.0     6197
2.0      5402
4.0      5332
3.0      5268
30.0     5121
26.0     4963
7.0      4905
28.0     4232
11.0     4131
18.0     3791
16.0     3768
14.0     3706
25.0     3432
40.0     3417
12.0     3412
15.0     3388
24.0     3190
22.0     2871
31.0     1802
38.0     1591
10.0     1560
37.0     1001
27.0      755
17.0      399
33.0      347
19.0      310
Name: analysis_neighborhoods, dtype: int64

The Department of Public Health and the Mayor’s Office of Housing and Community Development, with support from the Planning Department, created these 41 neighborhoods by grouping 2010 Census tracts, using common real estate and residents’ definitions for the purpose of providing consistency in the analysis and reporting of socio-economic, demographic, and environmental data, and data on City-funded programs and services.

Source: https://data.sfgov.org/Geographic-Locations-and-Boundaries/Analysis-Neighborhoods/p5b7-5n3h

In [51]:
reports.incident_subcategory.value_counts()

Larceny - From Vehicle                        61401
Other                                         44568
Larceny Theft - Other                         25585
Vandalism                                     19169
Simple Assault                                12664
                                              ...  
Robbery - Residential                            15
Bribery                                           5
Arrest                                            3
Human Trafficking, Involuntary Servitude          1
Human Trafficking (A), Commercial Sex Acts        1
Name: incident_subcategory, Length: 75, dtype: int64

In [66]:
reports['sf_find_neighborhoods'].value_counts()

32.0    32201
53.0    27392
20.0    19136
19.0    13967
21.0    10319
        ...  
68.0      137
63.0      133
7.0       104
69.0      102
70.0       49
Name: sf_find_neighborhoods, Length: 116, dtype: int64

These refer to Neighborhood boundaries that were defined in 2006 by the Mayor's Office of Neighborhood Services for use with the SF Find tool: http://propertymap.sfplanning.org/?name=sffind.

Source: https://data.sfgov.org/Geographic-Locations-and-Boundaries/SF-Find-Neighborhoods/pty2-tcw4

https://sfplanninggis.org/SFFind/?

In [24]:
reports['HSOC Zones as of 2018-06-05'].value_counts()

KeyError: 'HSOC Zones as of 2018-06-05'

HSOC refers to "Healthy Streets Operation Center", a San Francisco inter-agecy effort (SF Police Dept, Dept of Public Health, SF Public Works etc.) to coordinate the City's response to homelessness.

Source: http://hsh.sfgov.org/wp-content/uploads/HSOC-Presentation-for-LHCB-FINAL.pdf

In [ ]:
reports['OWED Public Spaces'].value_counts()

A list of public spaces being considered for management and activation by Office of Economic & Workforce Development (OEWD).

Source: https://data.sfgov.org/Geographic-Locations-and-Boundaries/OWED-Public-Spaces/gkqa-s74m

In [ ]:
reports['Central Market/Tenderloin Boundary Polygon - Updated'].value_counts()

This marks whethere or not the crime took place within the Central Market/Tenderloin district.

Source: https://data.sfgov.org/Geographic-Locations-and-Boundaries/Central-Market-Tenderloin-Boundary-Polygon/ywcr-44b8

In [ ]:
reports['Parks Alliance CPSI (27+TL sites)'].value_counts()

Boundaries of 27 open space sites that are part of the OEWD Citywide Public Space Initiative, plus UN Plaza, Boeddeker Park, TL Recreation Center, and Civic Center Plaza.

In this dataset, it marks which of those public spaces the crime had taken place in.

Source: https://data.sfgov.org/Geographic-Locations-and-Boundaries/Parks-Alliance-CPSI-27-TL-sites-/qjyb-yy3m

In [ ]:
reports['ESNCAG - Boundary File'].value_counts()

Marks whether the crime took place within the Embarcadero SAFE Navigation Center (ESNCAG) or not. 

Source: https://data.sfgov.org/dataset/ESNCAG-Boundary-File/8cs3-kxq7

https://sfmayor.org/article/embarcadero-safe-navigation-center-slated-open-end-year

In [ ]:
reports['Areas of Vulnerability, 2016'].value_counts()

These geographic designations were created to define geographic areas within San Francisco that have a higher density of vulnerable populations. These geographic designations will be used for the Health Care Services Master Plan and DPH's Community Health Needs Assessment.

Source: https://data.sfgov.org/Geographic-Locations-and-Boundaries/Areas-of-Vulnerability-2016/kc4r-y88d

In [ ]:
reports['Incident Code'].value_counts()

In [ ]:
sns.distplot(reports['Incident Code'])

In [ ]:
sns.distplot(reports['Incident Year'])

In [ ]:
reports['Incident Category'].value_counts()

In [ ]:
#sns.barplot(reports['Incident Category'])

In [ ]:
Next steps: impute null values sensibly 